**SECTION-1: UNDERSTANDING AND PREPARING THE RAW DATASET FOR ANALYSIS**

We obtained the dataset from the Gene Expression Omnibus (GEO) database.

Here's the link - https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE115828

It's important to understand what different types of GEO accessions (IDs) mean:
- A GEO accession pointing to the entire dataset from the experiment usually starts with 'GSE'.
- The accession for each sample within the experiment starts with 'GSM'.
- The accession describing the technology (illumina HiSeq in this case) used to generate the data starts with 'GPL'.

It is very important to understand which technology was used to generate the data. Transcriptomic data is mainly generated in two ways - microarray and RNA-seq. The data for this experiment (GSE115828) was generated using RNA-seq technology. The gene counts aren't directly generated from the sequencer. The raw output from the sequencer is analysed by specific tools that count the number of reads mapped to a specific gene or transcript. Refer to this paper for a detailed understanding of this:

*Dawadi, P., Pokharel, B., Shrestha, A., Niraula, D., Naeem, A., Miura, S., Roy, M., & Nepal, S. (2025). From bench to bytes: a practical guide to RNA sequencing data analysis. Frontiers in genetics, 16, 1697922. https://doi.org/10.3389/fgene.2025.1697922*

NOTE: Not all RNA-seq experiments provide raw counts. They might provide normalised counts in the form of FPKM or TPM. These normalised values cannot be used by DESeq2 to model the counts! I'll leave it to you as an exercise to figure out why that is.

Now that we have a (somewhat) clear idea of what this data is, we can move on to data processing - Before applying any python tool to model the raw counts data, we must convert the dataset (could be in .csv or .tsv/.txt format) to a pandas dataframe.

In [ ]:
# install and import necessary packages (colab usually has pandas and numpy pre-installed)

!pip install pydeseq2
import pandas as pd
import numpy as np
from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

In [ ]:
# step-1: Convert data to a pandas dataframe

# pandas will use the read_csv function to read the file and convert it to a pandas dataframe
df = pd.read_csv('gene_raw_counts.tsv', sep='\t')

# set the row numbers (or index) to GeneID
df = df.set_index('GeneID')

# transpose the dataframe to ensure genes are in the columns and samples are in the rows
counts_df = df.T

print(counts_df)

# print the sample names (we are just referencing the index of the dataframe - think of it like we are giving each row a name)
print(counts_df.index)

# print the gene names/IDs (we are just referencing the column names of the dataframe)
print(counts_df.columns)


GeneID      100287102  653635     102466751  107985730  100302278  645520     \
GSM3190962          3        510         17         48         12       2505   
GSM3190963          7       1002         36         85         16       3081   
GSM3190964          5        618         22         43         19       1354   
GSM3190965          6        367         19         60         11       1303   
GSM3190966          2        199          9         33         11        416   
...               ...        ...        ...        ...        ...        ...   
GSM3191480          4        126          5         35          8        688   
GSM3191481         11        271          7         26         11        652   
GSM3191482          5        728         22         50          8       1889   
GSM3191483          1        384          8         39         15       1121   
GSM3191484          5        350         10         37          9        685   

GeneID      79501      100996442  72973

**SECTION-2: METADATA GENERATION**

Some experiments do not have a readily available metadata file. It is up to you to figure out the experimental design used by the authors either by reading the paper (and supplementary information) or by parsing the SOFT files available in the GEO database. This step involves a lot of trial and error. Feel free to use AI prompts to parse the SOFT files. However, it's good to manually understand the internal structure of the SOFT file first so you can write better prompts.

In [ ]:
import os

file_soft = open('GSE115828_family.soft', 'r')
n=0
for line in file_soft.readlines():
  if n < 2000:
    if 'mgs' in line:
       if 'NA' not in line:
        if 'no grade' not in line:
          print(line.strip())
          print(f'for sample {n}')
          n += 1

# control = mgs_level: 1
# disease = mgs_level: 2, 3, 4

'''

Gemini prompt

The GSE115828_family.soft file contains metadata related to the GSE115828 accession. Write a python script that parses the soft file to extract the following line for each sample name:
!Sample_characteristics_ch1 = mgs_level:
and creates a separate dataset containing two columns - sample_name and disease_status, where the disease_status label 'control' is assigned if mgs_level is 1 else it is assigned 'AMD'

'''

!Sample_characteristics_ch1 = mgs_level: 2
for sample 0
!Sample_characteristics_ch1 = mgs_level: 2
for sample 1
!Sample_characteristics_ch1 = mgs_level: 3
for sample 2
!Sample_characteristics_ch1 = mgs_level: 2
for sample 3
!Sample_characteristics_ch1 = mgs_level: 3
for sample 4
!Sample_characteristics_ch1 = mgs_level: 2
for sample 5
!Sample_characteristics_ch1 = mgs_level: 2
for sample 6
!Sample_characteristics_ch1 = mgs_level: 4
for sample 7
!Sample_characteristics_ch1 = mgs_level: 4
for sample 8
!Sample_characteristics_ch1 = mgs_level: 4
for sample 9
!Sample_characteristics_ch1 = mgs_level: 1
for sample 10
!Sample_characteristics_ch1 = mgs_level: 4
for sample 11
!Sample_characteristics_ch1 = mgs_level: 3
for sample 12
!Sample_characteristics_ch1 = mgs_level: 2
for sample 13
!Sample_characteristics_ch1 = mgs_level: 2
for sample 14
!Sample_characteristics_ch1 = mgs_level: 2
for sample 15
!Sample_characteristics_ch1 = mgs_level: 3
for sample 16
!Sample_characteristics_ch1 = mgs_level: 

"\n\nGemini prompt\n\nThe GSE115828_family.soft file contains metadata related to the GSE115828 accession. Write a python script that parses the soft file to extract the following line for each sample name:\n!Sample_characteristics_ch1 = mgs_level:\nand creates a separate dataset containing two columns - sample_name and disease_status, where the disease_status label 'control' is assigned if mgs_level is 1 else it is assigned 'AMD'\n\n"

First, I'll parse the `GSE115828_family.soft` file to extract the `sample_name` and `mgs_level` for each entry. Then, based on the `mgs_level`, I'll assign the `disease_status` as 'control' or 'AMD'.

In [ ]:
# version 1

import pandas as pd

samples_data = []
current_sample_name = None

with open('GSE115828_family.soft', 'r') as file_soft:
    for line in file_soft:
        line = line.strip()
        if line.startswith('^SAMPLE = '):
            current_sample_name = line.split(' = ')[1]
        elif current_sample_name and 'mgs_level:' in line:
            try:
                # Extract the mgs_level number
                mgs_level_str = line.split('mgs_level:')[1].strip()
                mgs_level = int(mgs_level_str)

                # Determine disease_status
                disease_status = 'control' if mgs_level == 1 else 'AMD'

                samples_data.append({
                    'sample_name': current_sample_name,
                    'disease_status': disease_status
                })
                current_sample_name = None # Reset for the next sample
            except ValueError:
                # Handle cases where mgs_level might not be an integer or is missing
                print(f"Warning: Could not parse mgs_level from line: {line}")
                current_sample_name = None # Reset even if parsing fails for this mgs_level

# Create a pandas DataFrame from the collected data
sample_metadata_df = pd.DataFrame(samples_data)

# Display the first few rows of the created DataFrame
display(sample_metadata_df.head())

# Display the value counts for disease_status to verify distribution
display(sample_metadata_df['disease_status'].value_counts())

,sample_name,disease_status
0,GSM3190962,AMD
1,GSM3190963,AMD
2,GSM3190964,AMD
3,GSM3190965,AMD
4,GSM3190966,AMD


,count
disease_status,
AMD,390
control,127


In [ ]:
# version 2

import pandas as pd

samples_data = []
current_sample_name = None

with open('GSE115828_family.soft', 'r') as file_soft:
    for line in file_soft:
        line = line.strip()

        if line.startswith('^SAMPLE = '):
            current_sample_name = line.split(' = ')[1]

        elif current_sample_name and 'mgs_level:' in line:
            # Extract the raw string after 'mgs_level:'
            mgs_raw = line.split('mgs_level:')[1].strip().lower()

            # 1. Skip known exclusions
            if mgs_raw in ['na', 'no grade']:
                current_sample_name = None
                continue

            # 2. Handle specific levels and ranges
            if mgs_raw == '1':
                disease_status = 'control'
            elif any(val in mgs_raw for val in ['2', '3', '4', 'or']):
                # This catches '2', '3', '4', and the '2 or 3' range
                disease_status = 'AMD'
            else:
                # Fallback for unexpected strings
                print(f"Warning: Skipping unidentifiable mgs_level: {line}")
                current_sample_name = None
                continue

            samples_data.append({
                'sample_name': current_sample_name,
                'disease_status': disease_status
            })
            current_sample_name = None

# Create DataFrame
sample_metadata_df = pd.DataFrame(samples_data)

# Display Results
print("Value counts for disease_status:")
print(sample_metadata_df['disease_status'].value_counts())

Value counts for disease_status:
disease_status
AMD        391
control    127
Name: count, dtype: int64


In [ ]:
# save sample metadata

sample_metadata_df.to_csv('sample_metadata.csv', index=False)

In [ ]:
#sample_list = sample_metadata_df['sample_name'].unique().tolist()
#print(sample_list)

sample_metadata_df
metadata = sample_metadata_df.set_index('sample_name')
metadata

,disease_status
sample_name,
GSM3190962,AMD
GSM3190963,AMD
GSM3190964,AMD
GSM3190965,AMD
GSM3190966,AMD
...,...
GSM3191480,AMD
GSM3191481,AMD
GSM3191482,AMD


**SECTION-3: RUN DESEQ2**

Step-1: Start by filtering lowly expressed genes. We need to do this because genes with fewer than 10 reads are considered unreliable and thus assumed to be sequencing errors or artefacts.

Step-2: Ensure that both the metadata and the counts data have sample names as index.

Step-3: Now that the data is ready, follow the documentation to run DESeq2.

Here's the link - https://pydeseq2.readthedocs.io/en/stable/auto_examples/plot_minimal_pydeseq2_pipeline.html

In [ ]:
# filter very lowly expressed genes
genes_to_keep = counts_df.columns[counts_df.sum(axis=0) >= 10]
counts_df = counts_df[genes_to_keep]

Before creating the `DeseqDataSet`, I need to ensure that the sample names (index) in `counts_df` and `metadata` are perfectly aligned. I will filter `counts_df` to keep only the samples for which we have corresponding metadata.

In [ ]:
# Filter counts_df to include only samples present in metadata
common_samples = counts_df.index.intersection(metadata.index)
counts_df = counts_df.loc[common_samples]

# Filter metadata to include only samples present in counts_df (to ensure perfect alignment)
metadata = metadata.loc[common_samples]

# Verify that the indices are now aligned and have the same length
print(f"Number of samples in counts_df: {len(counts_df)}")
print(f"Number of samples in metadata: {len(metadata)}")
print(f"Are indices equal? {counts_df.index.equals(metadata.index)}")

Number of samples in counts_df: 516
Number of samples in metadata: 516
Are indices equal? True


In [ ]:
# step-2: creating a deseq2 object (deseq2 converts this dataframe into an anndata object)

inference = DefaultInference(n_cpus=8)
dds = DeseqDataSet(
    counts=counts_df,
    metadata=metadata,
    design="~disease_status",
    refit_cooks=True,
    inference=inference,
    # n_cpus=8, # n_cpus can be specified here or in the inference object
)

/usr/lib/python3.12/functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


In [ ]:
# this line fits the dispersions and LFCs (a bunch of stats)

dds.deseq2()

Fitting size factors...


Using None as control genes, passed at DeseqDataSet initialization


... done in 0.65 seconds.

Fitting dispersions...
... done in 75.85 seconds.

Fitting dispersion trend curve...
... done in 0.89 seconds.

Fitting MAP dispersions...
... done in 110.00 seconds.

Fitting LFCs...
... done in 46.35 seconds.

Calculating cook's distance...
... done in 1.83 seconds.

Replacing 291 outlier genes.

Fitting dispersions...
... done in 0.75 seconds.

Fitting MAP dispersions...
... done in 0.81 seconds.

Fitting LFCs...
... done in 0.48 seconds.



In [ ]:
ds = DeseqStats(dds, contrast=["disease_status", "AMD", "control"], inference=inference)
ds.summary()

results = ds.results_df
print(results)

results.to_csv('deseq2_results.txt', sep='\t')

Running Wald tests...
... done in 24.57 seconds.



Log2 fold change & Wald test p-value: disease_status AMD vs control
               baseMean  log2FoldChange     lfcSE      stat    pvalue  \
GeneID                                                                  
100287102      5.092575        0.016078  0.097038  0.165686  0.868404   
653635       340.128546        0.073387  0.084328  0.870257  0.384160   
102466751     10.641487        0.050246  0.091067  0.551745  0.581123   
107985730     37.467768       -0.021137  0.037813 -0.558978  0.576177   
100302278     11.360831       -0.028297  0.052599 -0.537981  0.590590   
...                 ...             ...       ...       ...       ...   
4541       16987.227641        0.009130  0.049748  0.183529  0.854383   
4556        1659.844181        0.011706  0.052513  0.222915  0.823601   
4519       85620.757712       -0.009840  0.038680 -0.254387  0.799196   
4576          61.896159       -0.005160  0.063013 -0.081880  0.934742   
4571        1172.924609        0.012171  0.053729  0.226